Workspace
=========

The core element of a project is the ``Workspace``. A project Workspace holds core information about the author, version and all entities stored in the `geoh5` file. It also knows how to create the core structure needed by [Geoscience ANALYST](https://mirageoscience.com/mining-industry-software/geoscience-analyst/) for visualization.

![workspace](./images/workspace.png)


## Open and close

You can either open an existing project or create a new project by simply entering the desired file name.

In [ ]:
from geoh5py.workspace import Workspace

# Create a new project
workspace = Workspace("my_project.geoh5")

Et voila!

![blankworkspace](./images/blankworkspace.png){width="50%"}

By default, the `*.geoh5` file is accessed in "read-write" mode. In the eventuality that the file is already used by Geoscience ANALYST, the mode gets changed to "read-only". This prevents users from modifying the file while used in an active session, but still allows them to extract data from the workspace. The same restriction does not apply to multiple python processes, as permitted by the `Single Writer Multiple Reader (SWMR)` feature of HDF5.

In [ ]:
print(workspace.geoh5)

After completing the read/write process, the workspace must be closed in order to release the file. Geoscience ANALYST does not allow reading on an opened file.

In [ ]:
workspace.close()

print(workspace._geoh5)

## Context manager

Likewise, a workspace can be accessed via a context manager which will handle closing the file at the of a process.

In [ ]:
with Workspace("my_project.geoh5") as workspace:
    print(workspace.geoh5)
    
print(workspace._geoh5)